In [7]:
import sys 
import numpy as np
import pandas as pd

sys.path.append(r'C:\Users\DerDo\Desktop\fantasy_basketball_project')
from scrape_functions import *
from database_conn import SQLiteClient

scraper = Scrape_Functions()
db_path = r"jordan.db"

In [2]:
jordan_season_avgs = scraper.player_avg_data("jordami01")
jordan_season_per36 = scraper.player_per36_data("jordami01")
jordan_season_sums = scraper.player_sum_data("jordami01")
jordan_season_advanced = scraper.player_advanced_data("jordami01")

In [8]:
COLUMN_RENAME_MAP = {
    "Season": "season",
    "Age": "age",
    "Team": "team",
    "Lg": "lg",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "FG": "fg",
    "FGA": "fga",
    "FG%": "fg_perc",
    "3P": "three_p",
    "3PA": "three_pa",
    "3P%": "three_p_perc",
    "2P": "two_p",
    "2PA": "two_pa",
    "2P%": "two_p_perc",
    "eFG%": "efg_perc",
    "FT": "ft",
    "FTA": "fta",
    "FT%": "ft_perc",
    "ORB": "orb",
    "DRB": "drb",
    "TRB": "trb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PF": "pf",
    "PTS": "pts",
    "Awards": "awards",
    "Season Type": "season_type"
}

# Rename columns explicitly
jordan_season_avgs = jordan_season_avgs.rename(columns=COLUMN_RENAME_MAP)

# Ensure only expected columns exist and are ordered correctly
EXPECTED_COLS = [
    "season", "age", "team", "lg", "pos",
    "g", "gs", "mp",
    "fg", "fga", "fg_perc",
    "three_p", "three_pa", "three_p_perc",
    "two_p", "two_pa", "two_p_perc",
    "efg_perc",
    "ft", "fta", "ft_perc",
    "orb", "drb", "trb",
    "ast", "stl", "blk",
    "tov", "pf", "pts",
    "awards", "season_type"
]

jordan_season_avgs = jordan_season_avgs[EXPECTED_COLS]

cols = {col: "TEXT NULL" for col in jordan_season_avgs.columns}
cols['season'] = "TEXT PRIMARY KEY"

# Create or use the DB
with SQLiteClient(db_path) as db:
    # 1) Create a table
    db.create_table(
        "jordan_season_avgs",
        cols
    )

    # Upsert dataframe
    db.upsert_dataframe(
        df=jordan_season_avgs, 
        table='jordan_season_avgs', 
        conflict_cols='season'
    )

    # Create the new table with the desired schema
    db.execute("""
        CREATE TABLE IF NOT EXISTS _jordan_season_avgs (
            season        TEXT,
            age           INTEGER,
            team          TEXT,
            lg            TEXT,
            pos           TEXT,
            g             INTEGER,
            gs            INTEGER,
            mp            REAL,
            fg            REAL,
            fga           REAL,
            fg_perc       REAL,
            three_p       REAL,
            three_pa      REAL,
            three_p_perc  REAL,
            two_p         REAL,
            two_pa        REAL,
            two_p_perc    REAL,
            efg_perc      REAL,
            ft            REAL,
            fta           REAL,
            ft_perc       REAL,
            orb           REAL,
            drb           REAL,
            trb           REAL,
            ast           REAL,
            stl           REAL,
            blk           REAL,
            tov           REAL,
            pf            REAL,
            pts           REAL,
            awards        TEXT,
            season_type   TEXT,
            PRIMARY KEY (season, team, season_type)
        );
    """)

    # Copy data from the old table into the new one
    db.execute("""
        INSERT OR REPLACE INTO _jordan_season_avgs
        SELECT
            season,
            age,
            team,
            lg,
            pos,
            g,
            gs,
            mp,
            fg,
            fga,
            fg_perc,
            three_p,
            three_pa,
            three_p_perc,
            two_p,
            two_pa,
            two_p_perc,
            efg_perc,
            ft,
            fta,
            ft_perc,
            orb,
            drb,
            trb,
            ast,
            stl,
            blk,
            tov,
            pf,
            pts,
            awards,
            season_type
        FROM jordan_season_avgs;
    """)

    # Swap tables
    db.execute("DROP TABLE jordan_season_avgs;")
    db.execute('ALTER TABLE _jordan_season_avgs RENAME TO jordan_season_avgs;')

    # Query the data
    data = db.query("SELECT * FROM jordan_season_avgs")



In [11]:
COLUMN_RENAME_MAP = {
    "Season": "season",
    "Age": "age",
    "Team": "team",
    "Lg": "lg",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "FG": "fg",
    "FGA": "fga",
    "FG%": "fg_perc",
    "3P": "three_p",
    "3PA": "three_pa",
    "3P%": "three_p_perc",
    "2P": "two_p",
    "2PA": "two_pa",
    "2P%": "two_p_perc",
    "eFG%": "efg_perc",
    "FT": "ft",
    "FTA": "fta",
    "FT%": "ft_perc",
    "ORB": "orb",
    "DRB": "drb",
    "TRB": "trb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PF": "pf",
    "PTS": "pts",
    "Awards": "awards",
    "Season Type": "season_type",
}

EXPECTED_COLS = [
    "season", "age", "team", "lg", "pos",
    "g", "gs", "mp",
    "fg", "fga", "fg_perc",
    "three_p", "three_pa", "three_p_perc",
    "two_p", "two_pa", "two_p_perc",
    "efg_perc",
    "ft", "fta", "ft_perc",
    "orb", "drb", "trb",
    "ast", "stl", "blk",
    "tov", "pf", "pts",
    "awards", "season_type"
]

jordan_season_per36 = jordan_season_per36.rename(columns=COLUMN_RENAME_MAP)
jordan_season_per36 = jordan_season_per36[EXPECTED_COLS]

cols = {col: "TEXT NULL" for col in jordan_season_per36.columns}
cols["season"] = "TEXT PRIMARY KEY"

with SQLiteClient(db_path) as db:
    db.create_table("jordan_season_per36", cols)

    db.upsert_dataframe(
        df=jordan_season_per36,
        table="jordan_season_per36",
        conflict_cols="season"
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _jordan_season_per36 (
            season        TEXT,
            age           INTEGER,
            team          TEXT,
            lg            TEXT,
            pos           TEXT,
            g             INTEGER,
            gs            INTEGER,
            mp            REAL,
            fg            REAL,
            fga           REAL,
            fg_perc       REAL,
            three_p       REAL,
            three_pa      REAL,
            three_p_perc  REAL,
            two_p         REAL,
            two_pa        REAL,
            two_p_perc    REAL,
            efg_perc      REAL,
            ft            REAL,
            fta           REAL,
            ft_perc       REAL,
            orb           REAL,
            drb           REAL,
            trb           REAL,
            ast           REAL,
            stl           REAL,
            blk           REAL,
            tov           REAL,
            pf            REAL,
            pts           REAL,
            awards        TEXT,
            season_type   TEXT,
            PRIMARY KEY (season, team, season_type)
        );
    """)

    db.execute("""
        INSERT OR REPLACE INTO _jordan_season_per36
        SELECT
            season,
            age,
            team,
            lg,
            pos,
            g,
            gs,
            mp,
            fg,
            fga,
            fg_perc,
            three_p,
            three_pa,
            three_p_perc,
            two_p,
            two_pa,
            two_p_perc,
            efg_perc,
            ft,
            fta,
            ft_perc,
            orb,
            drb,
            trb,
            ast,
            stl,
            blk,
            tov,
            pf,
            pts,
            awards,
            season_type
        FROM jordan_season_per36;
    """)

    db.execute("DROP TABLE jordan_season_per36;")
    db.execute("ALTER TABLE _jordan_season_per36 RENAME TO jordan_season_per36;")

    data = db.query("SELECT * FROM jordan_season_per36;")


In [12]:
pd.DataFrame(data)

,season,age,team,lg,pos,g,gs,mp,fg,fga,...,drb,trb,ast,stl,blk,tov,pf,pts,awards,season_type
0,1984-85,21,CHI,NBA,SG,4,4,171.0,7.2,16.4,...,3.4,4.8,7.2,2.3,0.8,3.2,3.2,24.6,,Playoff
1,1985-86,22,CHI,NBA,SG,3,3,135.0,12.8,25.3,...,3.7,5.1,4.5,1.9,1.1,3.7,3.5,34.9,,Playoff
2,1986-87,23,CHI,NBA,SG,3,3,128.0,9.8,23.6,...,3.9,5.9,5.1,1.7,2.0,2.3,3.1,30.1,,Playoff
3,1987-88,24,CHI,NBA,SG,10,10,427.0,11.6,21.9,...,4.0,6.0,4.0,2.0,0.9,3.3,3.2,30.6,,Playoff
4,1988-89,25,CHI,NBA,SG,17,17,718.0,10.0,19.6,...,4.7,6.0,6.5,2.1,0.7,3.4,3.3,29.6,,Playoff
5,1989-90,26,CHI,NBA,SG,16,16,674.0,11.7,22.8,...,4.9,6.1,5.8,2.4,0.7,3.0,2.9,31.4,,Playoff
6,1990-91,27,CHI,NBA,SG,17,17,689.0,10.3,19.6,...,4.7,5.6,7.4,2.1,1.2,2.2,2.8,27.6,Finals MVP-1,Playoff
7,1991-92,28,CHI,NBA,SG,22,22,920.0,11.3,22.7,...,3.9,5.4,5.0,1.7,0.6,3.2,2.4,29.7,Finals MVP-1,Playoff
8,1992-93,29,CHI,NBA,SG,19,19,783.0,11.5,24.3,...,4.4,5.9,5.2,1.8,0.8,2.1,2.7,30.6,Finals MVP-1,Playoff
9,1993-94,Did not play - retired/MiLB,,,,,,,,,...,,,,,,,,,,Regular


In [13]:
COLUMN_RENAME_MAP_SUMS = {
    "Season": "season",
    "Age": "age",
    "Team": "team",
    "Lg": "lg",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "FG": "fg",
    "FGA": "fga",
    "FG%": "fg_perc",
    "3P": "three_p",
    "3PA": "three_pa",
    "3P%": "three_p_perc",
    "2P": "two_p",
    "2PA": "two_pa",
    "2P%": "two_p_perc",
    "eFG%": "efg_perc",
    "FT": "ft",
    "FTA": "fta",
    "FT%": "ft_perc",
    "ORB": "orb",
    "DRB": "drb",
    "TRB": "trb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PF": "pf",
    "PTS": "pts",
    "Trp-Dbl": "trp_dbl",
    "Awards": "awards",
    "Season Type": "season_type",
}

EXPECTED_COLS_SUMS = [
    "season", "age", "team", "lg", "pos",
    "g", "gs", "mp",
    "fg", "fga", "fg_perc",
    "three_p", "three_pa", "three_p_perc",
    "two_p", "two_pa", "two_p_perc",
    "efg_perc",
    "ft", "fta", "ft_perc",
    "orb", "drb", "trb",
    "ast", "stl", "blk",
    "tov", "pf", "pts",
    "trp_dbl",
    "awards", "season_type"
]

jordan_season_sums = jordan_season_sums.rename(columns=COLUMN_RENAME_MAP_SUMS)
jordan_season_sums = jordan_season_sums[EXPECTED_COLS_SUMS]

cols = {col: "TEXT NULL" for col in jordan_season_sums.columns}
cols["season"] = "TEXT PRIMARY KEY"

with SQLiteClient(db_path) as db:
    db.create_table("jordan_season_sums", cols)

    db.upsert_dataframe(
        df=jordan_season_sums,
        table="jordan_season_sums",
        conflict_cols="season"
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _jordan_season_sums (
            season        TEXT,
            age           INTEGER,
            team          TEXT,
            lg            TEXT,
            pos           TEXT,
            g             INTEGER,
            gs            INTEGER,
            mp            INTEGER,
            fg            INTEGER,
            fga           INTEGER,
            fg_perc       REAL,
            three_p       INTEGER,
            three_pa      INTEGER,
            three_p_perc  REAL,
            two_p         INTEGER,
            two_pa        INTEGER,
            two_p_perc    REAL,
            efg_perc      REAL,
            ft            INTEGER,
            fta           INTEGER,
            ft_perc       REAL,
            orb           INTEGER,
            drb           INTEGER,
            trb           INTEGER,
            ast           INTEGER,
            stl           INTEGER,
            blk           INTEGER,
            tov           INTEGER,
            pf            INTEGER,
            pts           INTEGER,
            trp_dbl       INTEGER,
            awards        TEXT,
            season_type   TEXT,
            PRIMARY KEY (season, team, season_type)
        );
    """)

    db.execute("""
        INSERT OR REPLACE INTO _jordan_season_sums
        SELECT
            season,
            age,
            team,
            lg,
            pos,
            g,
            gs,
            mp,
            fg,
            fga,
            fg_perc,
            three_p,
            three_pa,
            three_p_perc,
            two_p,
            two_pa,
            two_p_perc,
            efg_perc,
            ft,
            fta,
            ft_perc,
            orb,
            drb,
            trb,
            ast,
            stl,
            blk,
            tov,
            pf,
            pts,
            trp_dbl,
            awards,
            season_type
        FROM jordan_season_sums;
    """)

    db.execute("DROP TABLE jordan_season_sums;")
    db.execute("ALTER TABLE _jordan_season_sums RENAME TO jordan_season_sums;")

    data = db.query("SELECT * FROM jordan_season_sums;")


In [14]:
pd.DataFrame(data)

,season,age,team,lg,pos,g,gs,mp,fg,fga,...,trb,ast,stl,blk,tov,pf,pts,trp_dbl,awards,season_type
0,1984-85,21,CHI,NBA,SG,4,4,171,34,78,...,23,34,11,4,15,15,117,0,,Playoff
1,1985-86,22,CHI,NBA,SG,3,3,135,48,95,...,19,17,7,4,14,13,131,0,,Playoff
2,1986-87,23,CHI,NBA,SG,3,3,128,35,84,...,21,18,6,7,8,11,107,0,,Playoff
3,1987-88,24,CHI,NBA,SG,10,10,427,138,260,...,71,47,24,11,39,38,363,0,,Playoff
4,1988-89,25,CHI,NBA,SG,17,17,718,199,390,...,119,130,42,13,68,65,591,1,,Playoff
5,1989-90,26,CHI,NBA,SG,16,16,674,219,426,...,115,109,45,14,56,54,587,0,,Playoff
6,1990-91,27,CHI,NBA,SG,17,17,689,197,376,...,108,142,40,23,43,53,529,0,Finals MVP-1,Playoff
7,1991-92,28,CHI,NBA,SG,22,22,920,290,581,...,137,127,44,16,81,62,759,0,Finals MVP-1,Playoff
8,1992-93,29,CHI,NBA,SG,19,19,783,251,528,...,128,114,39,17,45,58,666,1,Finals MVP-1,Playoff
9,1993-94,Did not play - retired/MiLB,,,,,,,,,...,,,,,,,,,,Regular


In [15]:
COLUMN_RENAME_MAP_ADV = {
    "Season": "season",
    "Age": "age",
    "Team": "team",
    "Lg": "lg",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "PER": "per",
    "TS%": "ts_perc",
    "3PAr": "three_par",
    "FTr": "ftr",
    "ORB%": "orb_perc",
    "DRB%": "drb_perc",
    "TRB%": "trb_perc",
    "AST%": "ast_perc",
    "STL%": "stl_perc",
    "BLK%": "blk_perc",
    "TOV%": "tov_perc",
    "USG%": "usg_perc",
    "OWS": "ows",
    "DWS": "dws",
    "WS": "ws",
    "WS/48": "ws_per_48",
    "OBPM": "obpm",
    "DBPM": "dbpm",
    "BPM": "bpm",
    "VORP": "vorp",
    "Awards": "awards",
    "Season Type": "season_type",
}

EXPECTED_COLS_ADV = [
    "season", "age", "team", "lg", "pos",
    "g", "gs", "mp",
    "per", "ts_perc", "three_par", "ftr",
    "orb_perc", "drb_perc", "trb_perc",
    "ast_perc", "stl_perc", "blk_perc",
    "tov_perc", "usg_perc",
    "ows", "dws", "ws", "ws_per_48",
    "obpm", "dbpm", "bpm", "vorp",
    "awards", "season_type"
]

jordan_season_advanced = jordan_season_advanced.rename(columns=COLUMN_RENAME_MAP_ADV)
jordan_season_advanced = jordan_season_advanced[EXPECTED_COLS_ADV]

cols = {col: "TEXT NULL" for col in jordan_season_advanced.columns}
cols["season"] = "TEXT PRIMARY KEY"

with SQLiteClient(db_path) as db:
    db.create_table("jordan_season_advanced", cols)

    db.upsert_dataframe(
        df=jordan_season_advanced,
        table="jordan_season_advanced",
        conflict_cols="season"
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _jordan_season_advanced (
            season      TEXT,
            age         INTEGER,
            team        TEXT,
            lg          TEXT,
            pos         TEXT,
            g           INTEGER,
            gs          INTEGER,
            mp          INTEGER,
            per         REAL,
            ts_perc     REAL,
            three_par   REAL,
            ftr         REAL,
            orb_perc    REAL,
            drb_perc    REAL,
            trb_perc    REAL,
            ast_perc    REAL,
            stl_perc    REAL,
            blk_perc    REAL,
            tov_perc    REAL,
            usg_perc    REAL,
            ows         REAL,
            dws         REAL,
            ws          REAL,
            ws_per_48   REAL,
            obpm        REAL,
            dbpm        REAL,
            bpm         REAL,
            vorp        REAL,
            awards      TEXT,
            season_type TEXT,
            PRIMARY KEY (season, team, season_type)
        );
    """)

    db.execute("""
        INSERT OR REPLACE INTO _jordan_season_advanced
        SELECT
            season,
            age,
            team,
            lg,
            pos,
            g,
            gs,
            mp,
            per,
            ts_perc,
            three_par,
            ftr,
            orb_perc,
            drb_perc,
            trb_perc,
            ast_perc,
            stl_perc,
            blk_perc,
            tov_perc,
            usg_perc,
            ows,
            dws,
            ws,
            ws_per_48,
            obpm,
            dbpm,
            bpm,
            vorp,
            awards,
            season_type
        FROM jordan_season_advanced;
    """)

    db.execute("DROP TABLE jordan_season_advanced;")
    db.execute("ALTER TABLE _jordan_season_advanced RENAME TO jordan_season_advanced;")

    data = db.query("SELECT * FROM jordan_season_advanced;")


In [20]:
career_teams = jordan_season_avgs[['season', 'team']]
career_teams['season'] = career_teams['season'].apply(lambda x: int(x[:x.find('-')]))

career_teams

C:\Users\DerDo\AppData\Local\Temp\ipykernel_29524\3870331999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  career_teams['season'] = career_teams['season'].apply(lambda x: int(x[:x.find('-')]))


,season,team
0,1984,CHI
1,1985,CHI
2,1986,CHI
3,1987,CHI
4,1988,CHI
5,1989,CHI
6,1990,CHI
7,1991,CHI
8,1992,CHI
9,1993,


In [25]:
jordan_team_1985 = scraper.team_avg_data('CHI', 1985)

In [26]:
jordan_team_1986 = scraper.team_avg_data('CHI', 1986)

In [29]:
jordan_team_1987 = scraper.team_avg_data('CHI', 1987)

In [30]:
jordan_team_1988 = scraper.team_avg_data('CHI', 1988)

In [31]:
jordan_team_1989 = scraper.team_avg_data('CHI', 1989)

In [32]:
jordan_team_1990 = scraper.team_avg_data('CHI', 1990)

In [33]:
jordan_team_1991 = scraper.team_avg_data('CHI', 1991)

In [34]:
jordan_team_1992 = scraper.team_avg_data('CHI', 1992)

In [35]:
jordan_team_1993 = scraper.team_avg_data('CHI', 1993)

In [38]:
jordan_team_1995 = scraper.team_avg_data('CHI', 1995)

In [40]:
jordan_team_1996 = scraper.team_avg_data('CHI', 1996)

In [41]:
jordan_team_1997 = scraper.team_avg_data('CHI', 1997)

In [42]:
jordan_team_1998 = scraper.team_avg_data('CHI', 1998)

In [46]:
jordan_team_2002 = scraper.team_avg_data('WAS', 2002)

In [48]:
jordan_team_2003 = scraper.team_avg_data('WAS', 2003)

In [55]:
for idx, table in enumerate([
    jordan_team_1985,
    jordan_team_1986,
    jordan_team_1987,
    jordan_team_1988,
    jordan_team_1989,
    jordan_team_1990,
    jordan_team_1991,
    jordan_team_1992,
    jordan_team_1993,
    jordan_team_1995,
    jordan_team_1996,
    jordan_team_1997,
    jordan_team_1996,
    jordan_team_2002,
    jordan_team_2003
]):
    year = idx + 2004 
    table['Year'] = year


In [58]:
jordan_team_data = pd.concat([
    jordan_team_1985,
    jordan_team_1986,
    jordan_team_1987,
    jordan_team_1988,
    jordan_team_1989,
    jordan_team_1990,
    jordan_team_1991,
    jordan_team_1992,
    jordan_team_1993,
    jordan_team_1995,
    jordan_team_1996,
    jordan_team_1997,
    jordan_team_1996,
    jordan_team_2002,
    jordan_team_2003
], axis=0)

In [59]:
jordan_team_data["player_year"] = (
    jordan_team_data["Player"].astype(str)
    + "_"
    + jordan_team_data["Year"].astype(str)
)

In [60]:
COLUMN_RENAME_MAP_TEAM = {
    "Rk": "rk",
    "Player": "player",
    "Age": "age",
    "Pos": "pos",
    "G": "g",
    "GS": "gs",
    "MP": "mp",
    "FG": "fg",
    "FGA": "fga",
    "FG%": "fg_perc",
    "3P": "three_p",
    "3PA": "three_pa",
    "3P%": "three_p_perc",
    "2P": "two_p",
    "2PA": "two_pa",
    "2P%": "two_p_perc",
    "eFG%": "efg_perc",
    "FT": "ft",
    "FTA": "fta",
    "FT%": "ft_perc",
    "ORB": "orb",
    "DRB": "drb",
    "TRB": "trb",
    "AST": "ast",
    "STL": "stl",
    "BLK": "blk",
    "TOV": "tov",
    "PF": "pf",
    "PTS": "pts",
    "Awards": "awards",
    "Year": "season_year",
    "player_year": "player_year"
}

EXPECTED_COLS_TEAM = [
    "rk",
    "player",
    "age",
    "pos",
    "g",
    "gs",
    "mp",
    "fg",
    "fga",
    "fg_perc",
    "three_p",
    "three_pa",
    "three_p_perc",
    "two_p",
    "two_pa",
    "two_p_perc",
    "efg_perc",
    "ft",
    "fta",
    "ft_perc",
    "orb",
    "drb",
    "trb",
    "ast",
    "stl",
    "blk",
    "tov",
    "pf",
    "pts",
    "awards",
    "season_year",
    "player_year"
]

# Rename columns
jordan_team_data = jordan_team_data.rename(columns=COLUMN_RENAME_MAP_TEAM)

# Create the composite key column (MUST exist before selecting EXPECTED_COLS_TEAM)
jordan_team_data["player_year"] = (
    jordan_team_data["player"].astype(str) + "_" + jordan_team_data["season_year"].astype(str)
)

# Enforce order
jordan_team_data = jordan_team_data[EXPECTED_COLS_TEAM]

# Staging schema
cols = {col: "TEXT NULL" for col in jordan_team_data.columns}
cols["player_year"] = "TEXT PRIMARY KEY"

with SQLiteClient(db_path) as db:
    db.create_table("jordan_team_data", cols)

    db.upsert_dataframe(
        df=jordan_team_data,
        table="jordan_team_data",
        conflict_cols="player_year"
    )

    db.execute("""
        CREATE TABLE IF NOT EXISTS _jordan_team_data (
            rk            INTEGER,
            player        TEXT,
            age           INTEGER,
            pos           TEXT,
            g             INTEGER,
            gs            INTEGER,
            mp            REAL,
            fg            REAL,
            fga           REAL,
            fg_perc       REAL,
            three_p       REAL,
            three_pa      REAL,
            three_p_perc  REAL,
            two_p         REAL,
            two_pa        REAL,
            two_p_perc    REAL,
            efg_perc      REAL,
            ft            REAL,
            fta           REAL,
            ft_perc       REAL,
            orb           REAL,
            drb           REAL,
            trb           REAL,
            ast           REAL,
            stl           REAL,
            blk           REAL,
            tov           REAL,
            pf            REAL,
            pts           REAL,
            awards        TEXT,
            season_year   INTEGER,
            player_year   TEXT,
            PRIMARY KEY (player_year)
        );
    """)

    db.execute("""
        INSERT OR REPLACE INTO _jordan_team_data
        SELECT
            rk,
            player,
            age,
            pos,
            g,
            gs,
            mp,
            fg,
            fga,
            fg_perc,
            three_p,
            three_pa,
            three_p_perc,
            two_p,
            two_pa,
            two_p_perc,
            efg_perc,
            ft,
            fta,
            ft_perc,
            orb,
            drb,
            trb,
            ast,
            stl,
            blk,
            tov,
            pf,
            pts,
            awards,
            season_year,
            player_year
        FROM jordan_team_data;
    """)

    db.execute("DROP TABLE jordan_team_data;")
    db.execute("ALTER TABLE _jordan_team_data RENAME TO jordan_team_data;")

    data = db.query("SELECT * FROM jordan_team_data;")


In [61]:
pd.DataFrame(data)

,rk,player,age,pos,g,gs,mp,fg,fga,fg_perc,...,trb,ast,stl,blk,tov,pf,pts,awards,season_year,player_year
0,1,Michael Jordan,21,SG,82,82,38.3,10.2,19.8,0.515,...,6.5,5.9,2.4,0.8,3.5,3.5,28.2,"MVP-6,ROY-1,AS,NBA2",2004,Michael Jordan_2004
1,2,Orlando Woolridge,25,SF,77,76,36.6,8.8,15.9,0.554,...,5.6,1.8,0.8,0.5,2.3,2.4,22.9,,2004,Orlando Woolridge_2004
2,3,Quintin Dailey,24,SG,79,0,26.6,6.6,14.1,0.473,...,2.6,2.4,0.9,0.1,1.9,2.4,16.0,6MOY-6,2004,Quintin Dailey_2004
3,4,Dave Corzine,28,C,82,50,25.1,3.4,6.9,0.486,...,5.1,1.7,0.4,0.8,1.5,2.3,8.5,,2004,Dave Corzine_2004
4,5,Dave Greenwood,27,PF,61,28,25.0,2.5,5.4,0.458,...,6.4,1.3,0.6,0.3,1.0,3.1,6.1,,2004,Dave Greenwood_2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,12,Etan Thomas,24,PF,38,0,13.5,1.6,3.3,0.492,...,4.3,0.1,0.2,0.6,0.9,1.7,4.8,,2018,Etan Thomas_2018
210,13,Charles Oakley,39,C,42,1,12.2,0.5,1.3,0.418,...,2.5,1.0,0.3,0.1,0.5,2.1,1.8,,2018,Charles Oakley_2018
211,14,Bobby Simmons,22,SG,36,2,10.5,1.2,3.1,0.393,...,2.1,0.6,0.3,0.1,0.2,1.4,3.3,,2018,Bobby Simmons_2018
212,15,Anthony Goldwire,31,PG,5,0,6.8,0.8,1.4,0.571,...,0.6,0.2,0.0,0.0,0.8,0.2,2.6,,2018,Anthony Goldwire_2018
